In [ ]:
import boto3
from PIL import Image
import logging
import os

def move_images(src_folder, dst_bucket):
    # Connect to S3
    s3 = boto3.resource('s3')
    dst = s3.Bucket(dst_bucket)

    # Create a logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    # Loop through all files in the source folder
    for filename in os.listdir(src_folder):
        file_path = os.path.join(src_folder, filename)
        # Check if the file is an image file
        if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
            try:
                # Open the image file
                img = Image.open(file_path)

                # Check if the image has no transparent pixels
                if not img.getbbox():
                    # Copy the image file to the destination bucket
                    dst.upload_file(file_path, filename)
                else:
                    # Log the image file that has transparent pixels
                    logger.info('{} has transparent pixels'.format(filename))
            except Exception as e:
                # Log the error
                logger.error('Error while processing {}: {}'.format(filename, e))

if __name__ == '__main__':
    import argparse

    # Parse the command line arguments
    parser = argparse.ArgumentParser(description='Move image files without transparent pixels from a local folder to an S3 bucket')
    parser.add_argument('src_folder', type=str, help='The path to the source folder')
    parser.add_argument('dst_bucket', type=str, help='The name of the destination S3 bucket')
    args = parser.parse_args()

    # Call the function to move the images
    move_images(args.src_folder, args.dst_bucket)
